In [954]:
# Импорт библиотек для проведения исследования с использованием модели GARCH (1,1)
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from arch import arch_model
from scipy import stats
import datetime as dt
import math
from scipy.stats import norm

In [955]:
# Импорт данных для обучающей выборки используя информационную систему Yahoo Finance
ticker = 'SPY' # Тикеры активов по которым загружаем данные
n_of_shares = ([100]) # Количество акций (шт.)
startDate = '2021-01-01' # Дата начала импорта данных
endDate = '2023-12-31' # Дата окончания импорта данных
df1 = yf.download(ticker, startDate, endDate) ['Close'] # Запрос данных через библиотеку Yahoo Finance

/var/folders/68/wtl2nd4s0j3fvb1d1nt0qxvh0000gn/T/ipykernel_26723/3931943355.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df1 = yf.download(ticker, startDate, endDate) ['Close'] # Запрос данных через библиотеку Yahoo Finance
[*********************100%***********************]  1 of 1 completed


In [956]:
#Расчёт портфельных метрик обучающей выборки
portfolio_value = np.sum(n_of_shares * np.array(df1.tail(1))) # Стоимость портфеля на последний импортированный день
portfolio_value_all = np.sum(n_of_shares * np.array(df1), axis=1) # Стоимости портфеля на конец каждого дня
weights = np.squeeze((np.array(df1.tail(1)) * n_of_shares)/portfolio_value) # Удельный вес каждого актива в портфеле на момент последнего дня
print(f'Стоимость портфеля на последний импортированный день -{portfolio_value: .2f} USD') # Вывод информации по стоимости портфеля
print(f'Удельный вес каждого актива в портфеле на момент последнего дня -{weights*100} %')# Вывод информации об удельных весах активов

Стоимость портфеля на последний импортированный день - 46521.39 USD
Удельный вес каждого актива в портфеле на момент последнего дня -100.0 %


In [957]:
#Расчёт статистических метрик портфеля обучающей выборки
portfolio_value_df = pd.DataFrame(portfolio_value_all) # Преобразование numpy array в pd dataframe
dailych = np.log(portfolio_value_df/portfolio_value_df.shift(1)).dropna() # Создание датафрейма с временными рядами дневных логнормальных изменений стоимости портфеля
portfolio_mean = dailych.mean().iloc[0] # Математическое ожидание дневного изменения стоимости портфеля
portfolio_std = dailych.std().iloc[0] # Дневная волатильность портфеля
portfolio_std_annual = (dailych.std().iloc[0]) * np.sqrt(252) # Аннуализированная волатильность портфеля за весь период импорта данных
print(f'Историческое математическое ожидание дневного изменения стоимости портфеля - {portfolio_mean*100: .3f} %')
print(f'Историческая дневная волатильность портфеля - {portfolio_std*100: .3f} %')
print(f'Историческая аннуализированная волатильность портфеля - {portfolio_std_annual*100: .3f} %')

Историческое математическое ожидание дневного изменения стоимости портфеля -  0.040 %
Историческая дневная волатильность портфеля -  1.109 %
Историческая аннуализированная волатильность портфеля -  17.607 %


In [958]:
# Имплементация модели GARCH(1,1)
model = arch_model(dailych, vol='Garch', p=1, q=1)
results = model.fit(disp='off')
print(results.summary())

                     Constant Mean - GARCH Model Results                      
Dep. Variable:                      0   R-squared:                       0.000
Mean Model:             Constant Mean   Adj. R-squared:                  0.000
Vol Model:                      GARCH   Log-Likelihood:                2390.33
Distribution:                  Normal   AIC:                          -4772.67
Method:            Maximum Likelihood   BIC:                          -4754.17
                                        No. Observations:                  752
Date:                Tue, Sep 30 2025   Df Residuals:                      751
Time:                        03:08:41   Df Model:                            1
                                 Mean Model                                 
                 coef    std err          t      P>|t|      95.0% Conf. Int.
----------------------------------------------------------------------------
mu         7.4616e-04  6.446e-06    115.752      0.000 [7.

/opt/anaconda3/lib/python3.12/site-packages/arch/univariate/base.py:309: DataScaleWarning: y is poorly scaled, which may affect convergence of the optimizer when
estimating the model parameters. The scale of y is 0.0001229. Parameter
estimation work better when this value is between 1 and 1000. The recommended
rescaling is 100 * y.

This warning can be disabled by either rescaling y before initializing the
model or by setting rescale=False.

  warnings.warn(


In [959]:
# Комментарий по результатам имплементации GARCH:
# alpha + beta: 0.98 < 1 (Условие стационарности выполняется)
# mu: 6.7975e-04 (t=88.022, p=0.000) (Статистически значим)
# omega: 2.5920e-06 (t=135,400, p=0.000) (Статистически значим)
# alpha[1]: 0.1000 (t=2.942, p=0.003) (Статистически значим)
# beta[1]: 0.8800 (t=28.150, p=0.000) (Статистически значим)

In [960]:
# Аналитика результатов по модели GARCH
half_life = np.log(0.5) / np.log(0.98)
print(f'Шоки волатильности сохраняются около{half_life: .2f} дней.')

Шоки волатильности сохраняются около 34.31 дней.


In [961]:
# Параметры прогноза временных рядов дневных доходностей
forecast_horizon = 1 # Горизонт прогнозирования
forecasts = results.forecast(horizon=forecast_horizon) # Прогнозирование временных рядов обученной моделью GARCH
volatility_forecast_df = np.sqrt(forecasts.variance.dropna()) # Расчёт дневной волатильности
mean_forecast_df = forecasts.mean # Экстракция данных о спрогнозированном среднем
forecast_horizon_mean = mean_forecast_df.iloc[-1] * forecast_horizon # Расчёт мат ожидания доходности за весь период прогнозирования
forecast_horizon_vol = volatility_forecast_df.iloc[-1] * np.sqrt(forecast_horizon) # Расчёт волатильности на весь горизонт прогнозирования для расчёта VaR
print(f'Спрогнозированное математическое ожидание дневного изменения стоимости портфеля {mean_forecast_df['h.1'].iloc[-1]*100: .3f} %')
print(f'Спрогнозированное математическое ожидание изменения стоимости портфеля за весь период {forecast_horizon_mean['h.1']*100: .3f} %')
print(f'Спрогнозированная дневная волатильность портфеля - {volatility_forecast_df['h.1'].iloc[-1]*100: .3f} %')
print(f'Спрогнозированная аннуализированная волатильность портфеля - {volatility_forecast_df['h.1'].iloc[-1] * np.sqrt(252)*100: .3f} %')

Спрогнозированное математическое ожидание дневного изменения стоимости портфеля  0.075 %
Спрогнозированное математическое ожидание изменения стоимости портфеля за весь период  0.075 %
Спрогнозированная дневная волатильность портфеля -  0.691 %
Спрогнозированная аннуализированная волатильность портфеля -  10.971 %


In [962]:
#Расчёт метрики Value at Risk основываясь на прогнозе GARCH(1,1)
cl = 0.99 # Уровень доверия
z_score = stats.norm.ppf(cl) # Расчёт z-score соответствующего уровню доверия
portfolio_value_VaR = portfolio_value + (portfolio_value * ((1+mean_forecast_df['h.1'].iloc[-1])**forecast_horizon)-portfolio_value)
VaR = portfolio_value_VaR * (forecast_horizon_vol * z_score - forecast_horizon_mean)
VaR = VaR['h.1']
print(f'Value at Risk для портфеля состоящего из активов: {ticker} , на уровне доверия {cl*100}% за период {forecast_horizon} торговых дней составляет{VaR: .2f} USD')

Value at Risk для портфеля состоящего из активов: SPY , на уровне доверия 99.0% за период 1 торговых дней составляет 713.75 USD


In [963]:
# Тестирование модели
# Импорт данных тестовой выборки
startDate_test = '2024-01-01' # Дата начала периода для тестовой выборки
endDate_test = '2025-01-01' # Дата окончания периода для тестовой выборки
test_df = yf.download(ticker, startDate_test, endDate_test)['Close'] # Импорт данных для тестовой выборки (суммарно 252 торговых дней)
test_df.describe() # Проверяем что суммарно в выборке 252 торговых дней

/var/folders/68/wtl2nd4s0j3fvb1d1nt0qxvh0000gn/T/ipykernel_26723/4114595078.py:5: FutureWarning: YF.download() has changed argument auto_adjust default to True
  test_df = yf.download(ticker, startDate_test, endDate_test)['Close'] # Импорт данных для тестовой выборки (суммарно 252 торговых дней)
[*********************100%***********************]  1 of 1 completed


Ticker,SPY
count,252.000000
mean,532.770824
std,37.997617
min,457.354462
25%,501.977509
50%,533.871979
75%,563.022522
max,600.509277


In [964]:
#Расчет портфельных метрик тестовой выборки
portfolio_value_test = np.sum(n_of_shares * np.array(test_df.tail(1))) # Стоимость портфеля на последний импортированный день тестовой выборки
portfolio_value_all_test = np.sum(n_of_shares * np.array(test_df), axis=1) # Стоимости портфеля на конец каждого дня в тестовой выборке
weights_test = np.squeeze((np.array(test_df.tail(1)) * n_of_shares)/portfolio_value_test) # Удельный вес каждого актива в портфеле на момент последнего дня тестовой выборки
print(f'Стоимость портфеля на последний импортированный день тестовой выборки -{portfolio_value_test: .2f} USD') # Вывод информации по стоимости портфеля
print(f'Удельный вес каждого актива в портфеле на момент последнего дня тестовой выборки - {weights_test*100} %')# Вывод информации об удельных весах активов

Стоимость портфеля на последний импортированный день тестовой выборки - 58098.91 USD
Удельный вес каждого актива в портфеле на момент последнего дня тестовой выборки - 100.0 %


In [965]:
#Расчёт статистических метрик тестовой выборки
portfolio_value_df_test = pd.DataFrame(portfolio_value_all_test) # Преобразование numpy array в pd dataframe
dailych_test = np.log(portfolio_value_df_test/portfolio_value_df_test.shift(1)).dropna() # Создание датафрейма с временными рядами дневных логнормальных изменений стоимости портфеля тестовой выборки
portfolio_mean_test = dailych_test.mean().iloc[0] # Математическое ожидание дневного изменения стоимости портфеля тестовой выборки
portfolio_std_test = dailych_test.std().iloc[0] # Дневная волатильность портфеля тестовой выборки
portfolio_std_annual_test = (dailych_test.std().iloc[0]) * np.sqrt(252) # Аннуализированная волатильность портфеля за весь период импорта данных тестовой выборки
print(f'Историческое математическое ожидание дневного изменения стоимости портфеля тестовой выборки - {portfolio_mean_test*100: .3f} %')
print(f'Историческая дневная волатильность портфеля тестовой выборки - {portfolio_std_test*100: .3f} %')
print(f'Историческая аннуализированная волатильность портфеля тестовой выборки - {portfolio_std_annual_test*100: .3f} %')

Историческое математическое ожидание дневного изменения стоимости портфеля тестовой выборки -  0.091 %
Историческая дневная волатильность портфеля тестовой выборки -  0.793 %
Историческая аннуализированная волатильность портфеля тестовой выборки -  12.593 %


In [966]:
# Расчёт Value at Risk для тестовой выборки
portfolio_std_test_VaR = portfolio_std_test * np.sqrt(forecast_horizon)
portfolio_mean_test_VaR = portfolio_mean_test * forecast_horizon
portfolio_value_test = portfolio_value_df_test.tail(1).iloc[0,0]
VaR_test = portfolio_value_test * (portfolio_std_test_VaR * z_score - portfolio_mean_test_VaR)
print(f'Value at Risk за период {forecast_horizon} д. тестовой выборки с уровнем доверия {cl*100}% составляет {VaR_test: .2f} USD')

Value at Risk за период 1 д. тестовой выборки с уровнем доверия 99.0% составляет  1019.47 USD


In [967]:
# Расчёт P/L по данным тестовой выборки
pl_df = (portfolio_value_df_test-portfolio_value_df_test.shift(1)).dropna()
loss_threshold = VaR
exceptions_n = (pl_df.iloc[:, 0] < -loss_threshold).sum()
print(f'Количество дней в тестовой выборке, в которых убыток превысил прогнозный {forecast_horizon} д. {cl*100}% VaR составляет {exceptions_n}')

Количество дней в тестовой выборке, в которых убыток превысил прогнозный 1 д. 99.0% VaR составляет 12


In [968]:
# Тестирование гипотезы на уровень точности модели (калибровка)
# H0 - Модель не недооценивает риск
# H1 - Модель недооценивает риск
alpha = 1-cl
z_statistic = exceptions_n-alpha*(252)/np.sqrt(alpha*(1-alpha)*252)
z_critical = stats.norm.ppf(1 - alpha)
if z_statistic <= -z_critical:
    print(f'H0 не отвергается, модель не недооценивает риск. Количество выбросов - {exceptions_n}.')
else:
    print(f'H0 отвергается, модель недооценивает риск. Количество выбросов - {exceptions_n}.')

H0 отвергается, модель недооценивает риск. Количество выбросов - 12.


In [973]:
# Саммари исследования
print("")
print("╔═══════════════════════════════════════════════════════╗")
print("║                 САММАРИ ИССЛЕДОВАНИЯ                 ║")
print("╚═══════════════════════════════════════════════════════╝")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("        ПОРТФЕЛЬНЫЕ МЕТРИКИ (ОБУЧАЮЩАЯ ВЫБОРКА)")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Стоимость портфеля: {portfolio_value:>15.2f} USD")
print(f"• Веса активов: {weights*100} %")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("       СТАТИСТИЧЕСКИЕ МЕТРИКИ (ОБУЧАЮЩАЯ ВЫБОРКА)")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Мат. ожидание (дневное): {portfolio_mean*100:>10.3f} %")
print(f"• Волатильность (дневная): {portfolio_std*100:>10.3f} %")
print(f"• Волатильность (годовая): {portfolio_std_annual*100:>10.3f} %")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("               ПРОГНОЗ GARCH(1,1)")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Прогноз мат. ожидания (день): {mean_forecast_df['h.1'].iloc[-1]*100:>8.3f} %")
print(f"• Прогноз мат. ожидания ({forecast_horizon} д.): {forecast_horizon_mean['h.1']*100:>5.3f} %")
print(f"• Прогноз волатильности (дневная): {volatility_forecast_df['h.1'].iloc[-1]*100:>6.3f} %")
print(f"• Прогноз волатильности (годовая): {volatility_forecast_df['h.1'].iloc[-1] * np.sqrt(252)*100:>6.3f} %")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("        ПОРТФЕЛЬНЫЕ МЕТРИКИ (ТЕСТОВАЯ ВЫБОРКА)")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Стоимость портфеля: {portfolio_value_test:>15.2f} USD")
print(f"• Веса активов: {weights_test*100} %")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("       СТАТИСТИЧЕСКИЕ МЕТРИКИ (ТЕСТОВАЯ ВЫБОРКА)")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Мат. ожидание (дневное): {portfolio_mean_test*100:>10.3f} %")
print(f"• Волатильность (дневная): {portfolio_std_test*100:>10.3f} %")
print(f"• Волатильность (годовая): {portfolio_std_annual_test*100:>10.3f} %")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("                 VaR ПРОГНОЗ")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Активы: {ticker}")
print(f"• Уровень доверия: {cl*100}%")
print(f"• Период: {forecast_horizon} дней")
print(f"• VaR: {VaR:>15.2f} USD")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("               VaR ТЕСТОВОЙ ВЫБОРКИ")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Период: {forecast_horizon} дней")
print(f"• Уровень доверия: {cl*100}%")
print(f"• VaR: {VaR_test:>15.2f} USD")
print("")

print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print("               ТЕСТИРОВАНИЕ МОДЕЛИ")
print("▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬")
print(f"• Количество выбросов: {exceptions_n}")
print(f"• Z-статистика: {z_statistic:.4f}")
print(f"• Критическое значение: {z_critical:.4f}")

if z_statistic > z_critical:
    print("• 🚨 РЕЗУЛЬТАТ: H0 ОТВЕРГАЕТСЯ")
    print("  Модель НЕДООЦЕНИВАЕТ риск")
else:
    print("• ✅ РЕЗУЛЬТАТ: H0 НЕ ОТВЕРГАЕТСЯ")
    print("  Модель адекватно оценивает риск")

print("")
print("╔═══════════════════════════════════════════════════════╗")
print("║                 ИССЛЕДОВАНИЕ ЗАВЕРШЕНО               ║")
print("╚═══════════════════════════════════════════════════════╝")


╔═══════════════════════════════════════════════════════╗
║                 САММАРИ ИССЛЕДОВАНИЯ                 ║
╚═══════════════════════════════════════════════════════╝

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
        ПОРТФЕЛЬНЫЕ МЕТРИКИ (ОБУЧАЮЩАЯ ВЫБОРКА)
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
• Стоимость портфеля:        46521.39 USD
• Веса активов: 100.0 %

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
       СТАТИСТИЧЕСКИЕ МЕТРИКИ (ОБУЧАЮЩАЯ ВЫБОРКА)
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
• Мат. ожидание (дневное):      0.040 %
• Волатильность (дневная):      1.109 %
• Волатильность (годовая):     17.607 %

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
               ПРОГНОЗ GARCH(1,1)
▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
• Прогноз мат. ожидания (день):    0.075 %
• Прогноз мат. ожидания (1 д.): 0.075 %
• Прогноз волатильности (дневная):  0.691 %
• Прогноз волатильности (годовая): 10.971 %

▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬▬
        ПОРТФЕЛЬНЫЕ МЕТРИКИ (ТЕС